In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import integrate

from control_theory import utils

## Problem parameters

In [ ]:
pend_mass = 0.127
cart_mass = 1.206
moment_of_inertia = 0.001
length = 0.178 * 2
K_f = 1.726
K_s = 4.487
B_c = 5.4
B_p = 0.002

## Getting A and b for our system

In [ ]:
A_0 = np.array([
    [pend_mass + cart_mass, -pend_mass * length],
    [-pend_mass * length, moment_of_inertia + pend_mass * length * length]
])

A_1 = np.array([
    [B_c, 0],
    [0, B_p]
])

A_2 = np.array([
    [0, 0],
    [0, -pend_mass * 9.8 * length]
])

print(A_0, A_1, A_2, sep='\n\n')

In [ ]:
first = -np.linalg.inv(A_0) @ A_2
second = -np.linalg.inv(A_0) @ A_1
third = np.linalg.inv(A_0) @ np.array([[1], [0]])

print(first, second, third, sep='\n\n')

### Finally we managed to get A and b for our system

In [ ]:
A = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [first[0][0], first[0][1], second[0][0], second[0][1]],
    [first[1][0], first[1][1], second[1][0], second[1][1]]
])

b = np.array([
    [0],
    [0],
    third[0],
    third[1]
])

print(A, b, sep='\n\n')

## Control synthesis

### Let's look at control matrix

In [ ]:
C = np.column_stack([
    b,
    A @ b,
    np.linalg.matrix_power(A, 2) @ b,
    np.linalg.matrix_power(A, 3) @ b
])

C, C.shape

In [ ]:
print(f"Control matrix rank: {np.linalg.matrix_rank(C)}")

In [ ]:
A_eigs = np.linalg.eigvals(A)
print(f"Eigs of the A: {A_eigs}")

### Here we get our control

In [ ]:
theta = (
    -np.array([0, 0, 0, 1])
    @ np.linalg.inv(C)
    @ utils.create_char_pol(-1, -2, A_eigs[2], A_eigs[3])(A)
)

print(f"Our controll: {theta}")

In [ ]:
# Before this moment all is identicall to maple

In [ ]:
def system(
        t: np.ndarray,
        x: np.ndarray,
        A_: np.ndarray,
        b_: np.ndarray,
        theta_: np.ndarray,
) -> np.ndarray:

    return (A_ + b_ @ theta_) @ x

In [ ]:
start, stop = 0, 5

time = np.linspace(start, stop, 300)
y_0 = np.array([0, 0.1, 0, 0])

sol = integrate.solve_ivp(system, (start, stop), y_0, dense_output=True, args=(A, b.T, theta), method="RK45")

In [ ]:
z = sol.sol(time)

In [ ]:
plt.plot(time, z[0])

In [ ]:
plt.plot(time, z[1])

In [ ]:
plt.plot(time, z[2])

In [ ]:
plt.plot(time, z[3])